In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.tokenize import sent_tokenize





with open ('C:/Users/ADMIN/Documents/mob.txt','r', encoding = 'utf-8') as file:
    bk = file.read()

sentences= sent_tokenize(bk)

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from spacytextblob.spacytextblob import SpacyTextBlob
import numpy as np
import spacy


nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("spacytextblob")
il = []
for s in sentences:
    doc = nlp(s)
    p = doc._.polarity
    if p > 0:
        il.append(1)
    else:
        il.append(0)

labels = np.array(il)
        

t = Tokenizer(num_words = 10000, oov_token = "<OOV>")
t.fit_on_texts(sentences)
sequence = t.texts_to_sequences(sentences)


In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


ml = max([len(seq) for seq in sequence])
ps = pad_sequences(sequence, maxlen = ml, padding = 'post')
X_train,X_test,y_train,y_test = train_test_split(ps,labels, test_size = 0.3, random_state = 42)



In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,GlobalMaxPooling1D,Bidirectional,Dropout

vs = len(t.word_index) + 1

model = Sequential([
    Embedding(input_dim = vs, output_dim = 128, input_length = ml),
    Bidirectional(LSTM(40, return_sequences = True)),
    GlobalMaxPooling1D(),
    Dense(units = 40, activation = 'relu'),
    Dropout(0.2),
    Dense(units = 1, activation = 'sigmoid')
])

model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 56, 128)           48768     
                                                                 
 bidirectional (Bidirectiona  (None, 56, 80)           54080     
 l)                                                              
                                                                 
 global_max_pooling1d (Globa  (None, 80)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 40)                3240      
                                                                 
 dropout (Dropout)           (None, 40)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 4

In [6]:


h = model.fit(X_train,y_train, epochs = 7, batch_size = 1, validation_data = (X_test,y_test), verbose = 2)

Epoch 1/7
25/25 - 34s - loss: 0.7009 - accuracy: 0.4000 - val_loss: 0.6944 - val_accuracy: 0.4545 - 34s/epoch - 1s/step
Epoch 2/7
25/25 - 1s - loss: 0.6810 - accuracy: 0.6000 - val_loss: 0.6909 - val_accuracy: 0.5455 - 1s/epoch - 41ms/step
Epoch 3/7
25/25 - 1s - loss: 0.6455 - accuracy: 0.6400 - val_loss: 0.6923 - val_accuracy: 0.5455 - 1s/epoch - 42ms/step
Epoch 4/7
25/25 - 1s - loss: 0.5703 - accuracy: 0.8400 - val_loss: 0.7005 - val_accuracy: 0.5455 - 1s/epoch - 40ms/step
Epoch 5/7
25/25 - 1s - loss: 0.2627 - accuracy: 1.0000 - val_loss: 0.8585 - val_accuracy: 0.7273 - 945ms/epoch - 38ms/step
Epoch 6/7
25/25 - 1s - loss: 0.0177 - accuracy: 1.0000 - val_loss: 1.2616 - val_accuracy: 0.7273 - 967ms/epoch - 39ms/step
Epoch 7/7
25/25 - 1s - loss: 0.0071 - accuracy: 1.0000 - val_loss: 0.9846 - val_accuracy: 0.8182 - 962ms/epoch - 38ms/step


In [7]:
def preproc(model,nt,t,max_len):
    sequences = t.texts_to_sequences([nt])
    padseq = pad_sequences(sequences, maxlen=ml, padding = "post")
    prediction = model.predict(padseq)
    return "Positive" if prediction  > 0.5 else "Negative"

In [8]:

nt = "shoot the Commie bastard"



sent = preproc(model,nt,t,ml)


In [9]:
sent

'Negative'